In [29]:
from Acesso import Login
from Query import Query
from Email import Email
from RemoverArquivo import Remover
from Moeda import Moeda
from Tempo import DataHora
import os
from glob import glob
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)


querys={

    'Receber':

    """

    SELECT * FROM netfeira.vw_contareceber
    ORDER BY [Data de Vencimento]    
        
    """,

    'Segmento':

    """

    SELECT * FROM netfeira.vw_segmento
    
    """,

    'Cliente':

    """

    SELECT * FROM netfeira.vw_cliente
    
    """

}

In [30]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [31]:
tabelas_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal'],
      dtype='object')

In [32]:
tabelas_df['Segmento'].columns

Index(['ID Segmento', 'Segmento', 'Canal'], dtype='object')

In [33]:
tabelas_df['Receber'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente', 'Razão Social',
       'Nome Fantasia', 'Matriz', 'ID Segmento', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$', 'Status do Título', 'Dias', 'Alerta'],
      dtype='object')

In [34]:
tabelas_df['Cliente']=tabelas_df['Cliente'].merge(tabelas_df['Segmento'],on='ID Segmento',how='inner')[['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'Segmento','Canal', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal']]

In [35]:
tabelas_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'Segmento', 'Canal', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal'],
      dtype='object')

In [36]:
tabelas_df['Receber']=tabelas_df['Receber'][['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$', 'Status do Título', 'Dias', 'Alerta']]

In [37]:
tabelas_df['Receber'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$', 'Status do Título', 'Dias', 'Alerta'],
      dtype='object')

In [38]:
tabelas_df['Receber']=tabelas_df['Receber'].merge(tabelas_df['Cliente'],on='ID Cliente',how='inner')[['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente','CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'Segmento', 'Canal', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$', 'Status do Título', 'Dias', 'Alerta']]

In [41]:
titulos_df=pd.DataFrame()

titulos_df=tabelas_df['Receber']

In [42]:
titulos_df['Saldo R$']=titulos_df.apply(lambda info: info['Valor Líquido']-info['Pago R$'],axis=1)

In [47]:
titulos_df=titulos_df.loc[titulos_df['Status do Título']=='VENCIDO']

In [52]:
grupo_df=pd.DataFrame()

grupo_df=titulos_df[['Alerta','Saldo R$']].groupby(['Alerta'],as_index=False).sum()

In [56]:
grupo_df['Título']=grupo_df['Alerta'].apply(lambda info: len(titulos_df['Título'].loc[titulos_df['Alerta']==info].unique().tolist()))

In [58]:
grupo_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

In [59]:
grupo_df

,Alerta,Saldo R$,Título
0,+ 90 DIAS,608696.72,441
1,6 A 15 DIAS,131953.09,291
2,ATÉ 5 DIAS,117208.92,182
3,16 A 30 DIAS,100322.29,295
4,31 A 60 DIAS,56377.63,110
5,61 A 90 DIAS,27302.35,46
